# Downloading using OpenID Connect (OIDC)

Requires a CDSE account

### Credentials
Username and password in a text file "cdse_credentials.txt" with the following structure:

```username=your_cdse_username
password=your_cdse_password```

In [10]:
def read_cdse_credentials(filepath="cdse_credentials.txt"):
    creds = {}
    with open(filepath, "r") as f:
        for line in f:
            key, value = line.strip().split("=", 1)
            creds[key.strip()] = value.strip()
    return creds["username"], creds["password"]

USERNAME, PASSWORD = read_cdse_credentials()

print("Credentials loaded (username only shown):", USERNAME)

Credentials loaded (username only shown): javieralonso.concha@esa.int


In [11]:
import requests

auth_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

data = {
    "client_id": "cdse-public",   # default public client
    "grant_type": "password",
    "username": USERNAME,
    "password": PASSWORD
}

response = requests.post(auth_url, data=data)
token = response.json()["access_token"]

print("Access token:", token[:60] + "...")


Access token: eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZK...


In [12]:
catalog_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
headers = {"Authorization": f"Bearer {token}"}

params = {
    "$filter": (
        "Collection/Name eq 'SENTINEL-2' "
        "and ContentDate/Start gt 2025-01-01T00:00:00.000Z "
        "and ContentDate/Start lt 2025-01-31T23:59:59.999Z "
        "and Attributes/OData.CSC.StringAttribute/any(a:a/Name eq 'productType' and a/Value eq 'S2MSI1C')"
    ),
    "$top": 5
}

resp = requests.get(catalog_url, headers=headers, params=params)
data = resp.json()["value"]

for p in data:
    print(p["Id"], p["Name"])


b48648ae-29f9-4491-acc4-32d9eaad21b0 S2B_MSIL1C_20250119T142709_N0511_R053_T21PVR_20250119T192320.SAFE
3e7473f3-af16-4829-949f-209bf2adbe15 S2B_MSIL1C_20250101T000619_N0511_R073_T56NPG_20250101T010230.SAFE
e351430b-38ea-4ea8-8d7a-a435f22480bd S2B_MSIL1C_20250101T000619_N0511_R073_T56NRG_20250101T010230.SAFE
2467d219-a97c-4533-b616-cb27ad6945ef S2B_MSIL1C_20250101T000619_N0511_R073_T57NTB_20250101T010230.SAFE
724ace31-18c3-4e52-b240-241173143c20 S2B_MSIL1C_20250101T000619_N0511_R073_T57NUA_20250101T010230.SAFE


### Add a filter based on an ROI

In [13]:
bbox = "POLYGON((2.25 48.80, 2.25 48.90, 2.40 48.90, 2.40 48.80, 2.25 48.80))"

params["$filter"] += f" and OData.CSC.Intersects(area=geography'SRID=4326;{bbox}')"

resp = requests.get(catalog_url, headers=headers, params=params)
data = resp.json()["value"]

for p in data:
    print(p["Id"], p["Name"])

e70a20f7-7c8e-485b-86a8-ac324fa6d66a S2B_MSIL1C_20250102T110349_N0511_R094_T31UDQ_20250102T143539.SAFE
306150e6-5d9d-40dc-bfde-7fdb1abba1e2 S2A_MSIL1C_20250104T105441_N0511_R051_T31UDQ_20250104T125200.SAFE
a4f5f471-eff8-4889-9b87-62015ad77384 S2B_MSIL1C_20250109T105329_N0511_R051_T31UDQ_20250109T124435.SAFE
12263b7f-78e6-4ea3-bbea-23c53eee7a28 S2B_MSIL1C_20250112T110329_N0511_R094_T31UDQ_20250112T131948.SAFE
c2698d12-939f-42d8-9e9e-8e42cc2c35a2 S2A_MSIL1C_20250114T105411_N0511_R051_T31UDQ_20250114T125009.SAFE


In [14]:
product_id = data[0]["Id"]

download_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"

with requests.get(download_url, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(f"./products/{data[0]['Name']}.zip", "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("✅ Download complete:", data[0]["Name"] + ".zip")


✅ Download complete: S2B_MSIL1C_20250102T110349_N0511_R094_T31UDQ_20250102T143539.SAFE.zip
